## Prompt engieneering



In [5]:
# %pip install ollama
from ollama import chat

response = chat(model='gemma3:1b', messages=[
  {
    'role': 'user',
    'content': 'Who ist the most pwerful character in Star Wars?',
  },
])
# notice your exact answer could be different due to some random processes in the model (learn about that later)
print(response.message.content)

That's a fantastic and endlessly debated question! There's no *single* definitive answer to who the most powerful character is in Star Wars, as "power" can be measured in different ways. However, **Grand Admiral Thrawn** is overwhelmingly considered the most powerful character in the Star Wars saga, and for good reason.

Here's a breakdown of why, and why other contenders exist:

**Why Thrawn is Considered the Most Powerful:**

* **Genius-Level Intelligence:** Thrawn is arguably the smartest being in the entire Star Wars universe. He possesses a deep understanding of strategy, history, and psychology that far surpasses even the most brilliant Jedi and Sith commanders.
* **Master Strategist:** He’s renowned for his incredibly complex and incredibly precise strategies. He can anticipate moves, identify weaknesses, and manipulate situations with unnerving accuracy. He’s often referred to as “The Strategist.”
* **Temporal Manipulation:** Thrawn’s ability to subtly influence events through 